# Troubleshooting Azure Package Installation Issues

This notebook demonstrates how to troubleshoot and resolve Python package installation errors, particularly with Azure AI packages and Docker builds. We'll walk through the issues encountered with `azure_ai_evaluation` package installation and provide solutions.

## 1. Analyzing Package Installation Errors

Let's start by analyzing the error message to understand the root cause of package installation failures.

In [ ]:
# First, let's install some utilities for parsing and analyzing error messages
!pip install packaging rich

In [ ]:
import re
import sys
from packaging import version
from rich import print
from rich.panel import Panel
from rich.console import Console
from rich.syntax import Syntax

console = Console()

In [ ]:
def parse_pip_error(error_message):
    """Parse a pip error message to extract key information"""
    results = {}

    # Extract package name and requested version
    package_match = re.search(r'requirement ([\w_-]+)==([\d\.\w]+)', error_message)
    if package_match:
        results['package_name'] = package_match.group(1)
        results['requested_version'] = package_match.group(2)

    # Extract available versions
    versions_match = re.search(r'\(from versions: ([^\)]+)\)', error_message)
    if versions_match:
        versions_str = versions_match.group(1)
        versions = [v.strip() for v in versions_str.split(',')]
        results['available_versions'] = versions

    return results

# Example error message from our deployment
error_message = """Downloading azure_ai_projects-1.0.0b11-py3-none-any.whl.metadata (23 kB)
ERROR: Could not find a version that satisfies the requirement azure_ai_evaluation==1.0.0b11 (from versions: 0.0.0b0, 1.0.0b1, 1.0.0b2, 1.0.0b3, 1.0.0b4, 1.0.0b5, 1.0.0, 1.0.1, 1.1.0, 1.2.0, 1.3.0, 1.4.0, 1.5.0, 1.6.0, 1.7.0, 1.8.0, 1.9.0, 1.10.0)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for azure_ai_evaluation==1.0.0b11
The command '/bin/sh -c pip install --no-cache-dir --upgrade -r requirements.txt' returned a non-zero code: 1
2025/08/15 13:34:59 Container failed during run: build. No retries remaining.
failed to run step ID: build: exit status 1
"""

parsed_error = parse_pip_error(error_message)
console.print(Panel.fit(
    "[bold red]Package Installation Error Analysis[/]\n\n"
    f"Package: [bold]{parsed_error.get('package_name', 'Unknown')}[/]\n"
    f"Requested version: [bold]{parsed_error.get('requested_version', 'Unknown')}[/]\n"
    f"Available versions: [green]{', '.join(parsed_error.get('available_versions', ['Unknown']))}[/]\n\n"
    "[bold yellow]Root Cause:[/] The requested beta version (1.0.0b11) is not available in the package repository."
))

### Understanding the Error

The error message indicates that:

1. We're trying to install `azure_ai_evaluation==1.0.0b11`
2. This specific beta version is not available in the PyPI repository
3. The available versions range from beta versions (0.0.0b0 through 1.0.0b5) to stable releases (1.0.0 through 1.10.0)
4. There's a Docker build failure because pip couldn't find the specified version

The most critical part of this error is that we're trying to install a specific beta version that's not available in the public repositories.

## 2. Package Version Management

Let's explore how to find available package versions and install a specific compatible version.

In [ ]:
import subprocess
import json

def get_package_versions(package_name):
    """Get available versions of a package from PyPI"""
    try:
        # Run pip index versions command
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'index', 'versions', package_name, '--format=json'],
            capture_output=True,
            text=True,
            check=True
        )
        data = json.loads(result.stdout)
        versions = data.get('versions', [])
        return versions
    except (subprocess.SubprocessError, json.JSONDecodeError) as e:
        print(f"Error fetching package versions: {e}")
        return []

def check_installed_package(package_name):
    """Check if a package is installed and get its version"""
    try:
        # Try to import the package
        __import__(package_name.replace('-', '_'))

        # Get the version
        result = subprocess.run(
            [sys.executable, '-c', f"import {package_name.replace('-', '_')}; print({package_name.replace('-', '_')}.__version__)"],
            capture_output=True,
            text=True,
            check=True
        )
        return result.stdout.strip()
    except (ImportError, subprocess.SubprocessError):
        return None

In [ ]:
# Let's check available versions for azure_ai_evaluation
package_name = "azure_ai_evaluation"

console.print(f"[bold]Available versions for {package_name}:[/]")
versions = get_package_versions(package_name)
if versions:
    # Separate into beta/preview and stable versions
    beta_versions = [v for v in versions if 'b' in v or 'a' in v or 'rc' in v]
    stable_versions = [v for v in versions if v not in beta_versions]

    console.print(f"[green]Stable versions:[/] {', '.join(stable_versions)}")
    console.print(f"[yellow]Beta/preview versions:[/] {', '.join(beta_versions)}")
else:
    console.print("[red]Failed to fetch versions[/]")

In [ ]:
# Now let's demonstrate how to install the latest stable version
print(f"Installing latest stable version of {package_name}...")

# This will actually install the package - uncomment if you want to run it
# !pip install "azure_ai_evaluation>=1.0.0" --upgrade

# For demonstration, show the pip command that would be run
console.print(Syntax("pip install \"azure_ai_evaluation>=1.0.0\" --upgrade", "bash", theme="monokai"))

# Check what version we have installed
installed_version = check_installed_package(package_name)
if installed_version:
    console.print(f"[green]Currently installed version:[/] {installed_version}")
else:
    console.print("[yellow]Package not currently installed[/]")

### Fixing the Requirements File

Here's how we fixed the `requirements.txt` file to avoid the version issue:

In [ ]:
original_requirements = """azure_ai_agents==1.0.0
azure_ai_projects==1.0.0b11
azure_ai_evaluation==1.0.0b11

azure-core==1.34.0  # other versions might not compatible"""

fixed_requirements = """azure_ai_agents>=1.0.0
azure_ai_projects>=1.0.0
azure_ai_evaluation>=1.10.0

azure-core>=1.34.0"""

console.print("[bold]Original requirements:[/]")
console.print(Syntax(original_requirements, "python", theme="monokai"))

console.print("[bold]Fixed requirements:[/]")
console.print(Syntax(fixed_requirements, "python", theme="monokai"))

console.print(Panel("[bold green]Key changes:[/]\n\n"
                   "1. Changed from pinned versions (==) to minimum versions (>=)\n"
                   "2. Removed specific beta version requirements\n"
                   "3. Updated to use latest stable release of azure_ai_evaluation (>=1.10.0)\n"
                   "4. Made azure-core version more flexible while maintaining minimum version"))

## 3. Using Package Alternatives

Sometimes you might need to find alternative packages or approaches when the exact version you want isn't available.

In [ ]:
def check_compatibility(package1, package2):
    """Check if two packages can be installed together"""
    try:
        # Run pip check with both packages
        cmd = f"{sys.executable} -m pip install {package1} {package2} --dry-run"
        result = subprocess.run(
            cmd,
            shell=True,
            capture_output=True,
            text=True
        )
        if result.returncode == 0:
            return True, "Compatible"
        else:
            return False, result.stderr
    except subprocess.SubprocessError as e:
        return False, str(e)

In [ ]:
# Let's check compatibility between different packages
console.print("[bold]Checking compatibility between packages:[/]")

package_pairs = [
    ("azure_ai_evaluation>=1.10.0", "azure_ai_projects>=1.0.0"),
    ("azure_ai_agents>=1.0.0", "azure_ai_evaluation>=1.10.0"),
    ("azure-core>=1.34.0", "azure_ai_evaluation>=1.10.0")
]

for pkg1, pkg2 in package_pairs:
    compatible, message = check_compatibility(pkg1, pkg2)
    if compatible:
        console.print(f"[green]✓[/] {pkg1} and {pkg2} are compatible")
    else:
        console.print(f"[red]✗[/] {pkg1} and {pkg2} have compatibility issues: {message}")

In [ ]:
# Alternative approach: use feature detection instead of version checking
code_example = """# Instead of relying on specific package versions,
# use feature detection to handle API differences

try:
    # Try importing from the newer API
    from azure.ai.evaluation import evaluate, AIAgentConverter

    # Use the newer API
    def run_evaluation(data, evaluators):
        return evaluate(
            evaluation_name="my-evaluation",
            data=data,
            evaluators=evaluators
        )
except ImportError:
    # Fall back to older API or alternative implementation
    from azure.ai.evaluation.legacy import evaluate_agent

    # Use the older API with a compatible interface
    def run_evaluation(data, evaluators):
        return evaluate_agent(
            name="my-evaluation",
            agent_data=data,
            evaluator_list=evaluators
        )
"""

console.print("[bold]Feature Detection Pattern:[/]")
console.print(Syntax(code_example, "python", theme="monokai"))

## 4. Fixing Docker Container Build Issues

Now let's look at how to fix Docker container build issues related to package installation.

In [ ]:
# Example of a problematic Dockerfile
problematic_dockerfile = """FROM python:3.11-slim

WORKDIR /app

COPY requirements.txt .

# Problematic line - fails if any package can't be installed
RUN pip install --no-cache-dir --upgrade -r requirements.txt

COPY . .

CMD ["python", "-m", "uvicorn", "api.main:create_app", "--factory", "--host", "0.0.0.0", "--port", "8000"]
"""

# Improved Dockerfile with better error handling
improved_dockerfile = """FROM python:3.11-slim

WORKDIR /app

# Update pip first to avoid issues with older pip versions
RUN pip install --no-cache-dir --upgrade pip

# Copy requirements file
COPY requirements.txt .

# Install dependencies with better error handling
# Use --use-pep517 for modern build system
RUN pip install --no-cache-dir --use-pep517 --upgrade -r requirements.txt || \
    (echo "Package installation failed. Checking requirements..." && \
     pip install --no-cache-dir --use-pep517 --upgrade -r requirements.txt --verbose && \
     exit 1)

COPY . .

# Validate installation before starting
RUN python -c "import azure.ai.evaluation; print(f'Using azure.ai.evaluation version: {azure.ai.evaluation.__version__}')"

CMD ["python", "-m", "uvicorn", "api.main:create_app", "--factory", "--host", "0.0.0.0", "--port", "8000"]
"""

console.print("[bold]Problematic Dockerfile:[/]")
console.print(Syntax(problematic_dockerfile, "dockerfile", theme="monokai"))

console.print("[bold]Improved Dockerfile:[/]")
console.print(Syntax(improved_dockerfile, "dockerfile", theme="monokai"))

In [ ]:
# Example function to validate requirements.txt before Docker build
def validate_requirements_file(requirements_path):
    """Validate all packages in a requirements.txt file"""
    issues = []

    try:
        with open(requirements_path, 'r') as f:
            lines = f.readlines()

        for line_num, line in enumerate(lines, 1):
            # Skip empty lines and comments
            line = line.strip()
            if not line or line.startswith('#'):
                continue

            # Check for exact pinned versions
            if "==" in line and not any(x in line for x in [">", "<", "~="]):
                package = line.split("==")[0].strip()
                version = line.split("==")[1].strip()

                # Check if this version exists
                available_versions = get_package_versions(package)
                if version not in available_versions:
                    issues.append({
                        "line": line_num,
                        "package": package,
                        "version": version,
                        "issue": "Version not found in PyPI",
                        "suggestion": f"Use a minimum version (>={available_versions[-1]}) instead of exact version"
                    })

        return issues
    except Exception as e:
        return [{"line": 0, "package": "N/A", "version": "N/A", "issue": f"Error reading file: {str(e)}", "suggestion": "Check file path and permissions"}]

# Create a sample requirements.txt file for testing
sample_requirements = """fastapi==0.115.13
uvicorn[standard]==0.29.0
azure_ai_agents==1.0.0
azure_ai_projects==1.0.0b11  # This beta version might cause issues
azure_ai_evaluation==1.0.0b11  # This specific beta version doesn't exist
azure-core==1.34.0
"""

import tempfile
with tempfile.NamedTemporaryFile(mode='w', suffix='.txt', delete=False) as f:
    f.write(sample_requirements)
    temp_req_path = f.name

# Validate the requirements file
console.print("[bold]Validating requirements.txt file:[/]")
issues = validate_requirements_file(temp_req_path)

if issues:
    console.print("[red]Issues found:[/]")
    for issue in issues:
        console.print(f"Line {issue['line']}: [bold]{issue['package']}=={issue['version']}[/]")
        console.print(f"  Issue: [red]{issue['issue']}[/]")
        console.print(f"  Suggestion: [green]{issue['suggestion']}[/]")
else:
    console.print("[green]No issues found[/]")

## 5. Updating pip Package Manager

Updating pip can sometimes resolve package installation issues, especially with newer packages that require modern installation features.

In [ ]:
def check_pip_version():
    """Check the current version of pip"""
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pip', '--version'],
            capture_output=True,
            text=True,
            check=True
        )
        version_match = re.search(r'pip ([\d\.]+)', result.stdout)
        if version_match:
            return version_match.group(1)
        return "Unknown"
    except subprocess.SubprocessError:
        return "Error"

def check_pip_upgrade_available():
    """Check if a pip upgrade is available"""
    try:
        result = subprocess.run(
            [sys.executable, '-m', 'pip', 'install', '--upgrade', 'pip', '--dry-run'],
            capture_output=True,
            text=True
        )

        if "would be upgraded" in result.stdout or "Would install" in result.stdout:
            version_match = re.search(r'pip-([\d\.]+)', result.stdout)
            if version_match:
                return True, version_match.group(1)
            return True, "newer version"

        return False, None
    except subprocess.SubprocessError:
        return False, None

# Check current pip version
current_version = check_pip_version()
console.print(f"Current pip version: [bold]{current_version}[/]")

# Check if upgrade is available
upgrade_available, new_version = check_pip_upgrade_available()
if upgrade_available:
    console.print(f"Upgrade available to version: [bold green]{new_version}[/]")
    console.print("\nTo upgrade pip, run:")
    console.print(Syntax("pip install --upgrade pip", "bash", theme="monokai"))
else:
    console.print("[green]You have the latest version of pip[/]")

In [ ]:
# Script to automate pip updates in CI/CD pipelines
pip_update_script = """#!/bin/bash
# Script to ensure pip is up to date before installing packages

# Get current pip version
CURRENT_VERSION=$(pip --version | grep -oP 'pip \K[0-9.]+' || echo "unknown")
echo "Current pip version: $CURRENT_VERSION"

# Upgrade pip
echo "Upgrading pip..."
pip install --upgrade pip

# Verify upgrade
NEW_VERSION=$(pip --version | grep -oP 'pip \K[0-9.]+' || echo "unknown")
echo "New pip version: $NEW_VERSION"

# Now install requirements
echo "Installing requirements..."
pip install -r requirements.txt

# Print summary of installed packages
echo "\nInstalled packages:"
pip list | grep -E 'azure|fastapi|uvicorn'
"""

console.print("[bold]Automated pip update script for CI/CD:[/]")
console.print(Syntax(pip_update_script, "bash", theme="monokai"))

## Summary of Solutions for Package Installation Issues

Here's a summary of the solutions we explored for the package installation issues:

In [ ]:
console.print(Panel("[bold green]Solutions for Package Installation Issues[/]\n\n"
                   "1. [bold]Use minimum version specification[/] (>=1.0.0) instead of exact versions (==1.0.0b11)\n\n"
                   "2. [bold]Check available versions[/] before specifying requirements\n\n"
                   "3. [bold]Upgrade pip[/] before installing packages\n\n"
                   "4. [bold]Implement error handling[/] in Docker builds\n\n"
                   "5. [bold]Use feature detection[/] instead of version checking when possible\n\n"
                   "6. [bold]Validate requirements files[/] before building containers\n\n"
                   "7. [bold]Test package compatibility[/] between different Azure packages"
                  ))

The main solution for our specific issue was updating the `requirements.txt` file to use the latest stable releases of the Azure AI packages instead of the specific beta versions that are no longer available.